# SmartProjectionPoseFactor

`SmartProjectionPoseFactor<CALIBRATION>` is a "smart" factor optimized for visual SLAM problems where the **camera calibration is known and fixed**, and only the camera **poses** are being optimized.
It inherits from `SmartProjectionFactor` but simplifies the setup by taking a single shared `CALIBRATION` object (e.g., `Cal3_S2`, `Cal3Bundler`) assumed to be used by all cameras observing the landmark.

Key characteristics:
- **Implicit Point:** Like its base class, the 3D point is not an explicit variable.
- **Fixed Calibration:** Assumes a single, known calibration `K` for all measurements.
- **Pose Variables:** The factor connects `Pose3` variables directly.
- **`Values` Requirement:** Requires `Pose3` objects in the `Values` container for its keys.
- **Configuration:** Behavior is controlled by `SmartProjectionParams`.

**Use Case:** Ideal for visual SLAM or SfM when camera intrinsics are pre-calibrated and assumed constant.
**Alternative:** If calibration also needs optimization, use `SmartProjectionFactor`.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/slam/doc/SmartProjectionPoseFactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install --quiet gtsam-develop

In [1]:
import gtsam
import numpy as np
from gtsam import (Values, Point2, Point3, Pose3, Rot3, NonlinearFactorGraph,
                   SmartProjectionParams, SmartProjectionPoseFactorCal3_S2,
                   Cal3_S2)
from gtsam import symbol_shorthand
import graphviz

X = symbol_shorthand.X # Key for Pose3 variable

ImportError: cannot import name 'SmartProjectionPoseFactorCal3_S2' from 'gtsam' (c:\Users\porte\miniconda3\envs\gtsam\Lib\site-packages\gtsam\__init__.py)

## Creating and Adding Measurements

1. Create the factor with a noise model, the shared calibration `K`, and parameters.
2. Add measurements (2D points) and the corresponding pose keys.

In [1]:
smart_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)
smart_params = SmartProjectionParams() # Use default params
K = gtsam.make_shared_Cal3_S2(500, 500, 0, 320, 240) # Fixed calibration

# Factor type includes the Calibration type
smart_factor = SmartProjectionPoseFactorCal3_S2(smart_noise, K, smart_params)

# Add measurements and keys (Pose keys)
smart_factor.add(Point2(150, 505), X(0))
smart_factor.add(Point2(470, 495), X(1))
smart_factor.add(Point2(480, 150), X(2))

print(f"Smart factor involves {smart_factor.size()} measurements.")
smart_factor.print("SmartFactorPose: ")

Smart factor involves 3 measurements.
SmartFactorPose: SmartProjectionPoseFactor, z = 
 SmartProjectionFactor
linearizationMode: 0
triangulationParameters:
rankTolerance = 1e-09
enableEPI = false
landmarkDistanceThreshold = -1
dynamicOutlierRejectionThreshold = -1


result:
Degenerate

SmartFactorBase, z = 
measurement 0, px = 
[150; 505]

noise model = diagonal sigmas [1; 1];
measurement 1, px = 
[470; 495]

noise model = diagonal sigmas [1; 1];
measurement 2, px = 
[480; 150]

noise model = diagonal sigmas [1; 1];
Factor on x0 x1 x2


## Evaluating the Error

The `.error(values)` method uses the `Pose3` objects from `values` and the fixed `K` to triangulate the point and compute the error.

In [2]:
# Create Values containing Pose3 objects
values = Values()
pose0 = Pose3(Rot3.Ypr(0.1, -0.1, 0.2), Point3(-1, 0, 0.5))
pose1 = Pose3(Rot3.Ypr(0.0,  0.1, 0.1), Point3( 1, 0, 0.5))
pose2 = Pose3(Rot3.Ypr(-0.1, 0.0, -0.2), Point3( 0, 1, 0.5))

values.insert(X(0), pose0)
values.insert(X(1), pose1)
values.insert(X(2), pose2)

# Triangulate first to see the implicit point
point_result = smart_factor.point(values)
print(f"Triangulated point result:\n{point_result}")

if point_result.valid():
   # Calculate error
   total_error = smart_factor.error(values)
   print(f"\nTotal reprojection error (0.5 * sum(err^2/sigma^2)): {total_error:.4f}")
else:
   print("\nTriangulation failed, error calculation depends on degeneracyMode.")

Triangulated point result:
Valid triangulation with point [4.15063506 0.18267612 5.20423168]

Total reprojection error (0.5 * sum(err^2/sigma^2)): 103876.2153


## Linearization

Linearization works similarly to `SmartProjectionFactor`, producing a linear factor (default: Hessian) connecting the `Pose3` variables.

In [3]:
graph = NonlinearFactorGraph()
graph.add(smart_factor)

# Linearize (default mode is HESSIAN)
linear_factor = smart_factor.linearize(values)

if linear_factor:
    print("\nLinearized Factor (showing HessianFactor structure):")
    # Cast to HessianFactor to print its details
    # Note: The factor dimension is Pose3::dimension (6)
    hessian_factor = gtsam.RegularHessianFactorPose3.Downcast(linear_factor)
    if hessian_factor:
         hessian_factor.print()
    else:
         print("Linearized factor is not a HessianFactor (check params.linearizationMode)")
else:
    print("Linearization failed (likely due to triangulation degeneracy)")


Linearized Factor (showing HessianFactor structure):
RegularHessianFactor(6): density=100% keys={ x0 x1 x2 }
Augmented information matrix: (dimensions: 6, 6, 6) : 
{
	[    1900,     4864,     -698,    -5014,    -4686,     -552; ]
	[    4864,  1.337e+04,     1473, -9.522e+03, -1.623e+04,      447; ]
	[    -698,     1473,     4803,     3085,    -3660, -1.118e+04; ]
	[   -5014, -9.522e+03,     3085,     1318,     2669,    -1843; ]
	[   -4686, -1.623e+04,    -3660,     2669,     4137,    -1067; ]
	[    -552,      447, -1.118e+04,    -1843,    -1067,     4035; ]
	[    1318,     2669,    -1843,      768,    -2084,      849; ]
	[    2669,     4137,    -1067,    -2084,     6054,    -1412; ]
	[   -1843,    -1067,     4035,      849,    -1412,     4392; ]
	[    6273, -1.411e+04,    -2161,    11515,    16951,      996; ]
	[    -141,    -1903, -1.299e+04,      230,    -2148,     7017; ]
	[    -104,     5192,    -1500,     -373,     -620,    -7830; ]
	[    -830,     6917,     -785,     -395,    -1